# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
from tabulate import tabulate
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/coit/Workspace/udacity/data_engineering_nanodegree/data_modeling_with_apache_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Create cluster

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1']) # make a connection to a Cassandra instance on your local machine
    session = cluster.connect()
except Exception as exception:
    print(exception)

('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(61, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})


#### Create Keyspace

In [6]:
# Create keyspace

try:
    session.execute(
        """
        CREATE KEYSPACE IF NOT EXISTS sparkify_db1
        WITH REPLICATION =
            { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
        """
    )
except Exception as exception:
    print(exception)

#### Set Keyspace

In [7]:
# Set keyspace

try:
    session.set_keyspace('sparkify_db1')
except Exception as exception:
    print(exception)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# QUERY 1
# Provide the name of the artist, song title, and song's length for sessionId = 338, itemInSession = 4

# Create table

query = ("""
CREATE TABLE IF NOT EXISTS song_from_session_item (
    session_id       INT,
    item_in_session  INT,
    artist_name      VARCHAR,
    song_title       VARCHAR,
    song_length      DOUBLE,
    PRIMARY KEY (session_id, item_in_session)
    )
""")
try:
    session.execute(query)
except Exception as exception:
    print(exception)                    

In [9]:
# QUERY 1
# Provide the name of the artist, song title, and song's length for sessionId = 338, itemInSession = 4

# Insert data

filepath = 'event_datafile_new.csv'

with open(filepath, encoding = 'utf8') as file:
    csvreader = csv.reader(file)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_from_session_item (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# QUERY 1
# Provide the name of the artist, song title, and song's length for sessionId = 338, itemInSession = 4

# Verify data

query = "SELECT artist_name, song_title, song_length FROM song_from_session_item WHERE session_id = 338 AND item_in_session = 4"
try:
    query_df = pd.DataFrame(list(session.execute(query)))
except Exception as exception:
    print(exception)

print(tabulate(query_df, headers='keys', tablefmt='fancy_grid', showindex=False))

╒═══════════════╤═════════════════════════════════╤═══════════════╕
│ artist_name   │ song_title                      │   song_length │
╞═══════════════╪═════════════════════════════════╪═══════════════╡
│ Faithless     │ Music Matters (Mark Knight Dub) │       495.307 │
╘═══════════════╧═════════════════════════════════╧═══════════════╛


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
# QUERY 2
# Provide the name of the artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# Create table

query = ("""
CREATE TABLE IF NOT EXISTS song_from_user_session (
    user_id          INT,
    session_id       INT,
    item_in_session  INT,
    artist_name      VARCHAR,
    song_title       VARCHAR,
    user_first_name  VARCHAR,
    user_last_name   VARCHAR,
    PRIMARY KEY ((user_id, session_id), item_in_session)
    )
""")
try:
    session.execute(query)
except Exception as exception:
    print(exception)                    

In [12]:
# QUERY 2
# Provide the name of the artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# Insert data

filepath = 'event_datafile_new.csv'

with open(filepath, encoding = 'utf8') as file:
    csvreader = csv.reader(file)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_from_user_session (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# QUERY 2
# Provide the name of the artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# Verify data

query = "SELECT artist_name, song_title, user_first_name, user_last_name FROM song_from_user_session WHERE user_id = 10 AND session_id = 182"
try:
    query_df = pd.DataFrame(list(session.execute(query)))
except Exception as exception:
    print(exception)

print(tabulate(query_df, headers='keys', tablefmt='fancy_grid', showindex=False))

╒═══════════════════╤══════════════════════════════════════════════════════╤═══════════════════╤══════════════════╕
│ artist_name       │ song_title                                           │ user_first_name   │ user_last_name   │
╞═══════════════════╪══════════════════════════════════════════════════════╪═══════════════════╪══════════════════╡
│ Down To The Bone  │ Keep On Keepin' On                                   │ Sylvie            │ Cruz             │
├───────────────────┼──────────────────────────────────────────────────────┼───────────────────┼──────────────────┤
│ Three Drives      │ Greece 2000                                          │ Sylvie            │ Cruz             │
├───────────────────┼──────────────────────────────────────────────────────┼───────────────────┼──────────────────┤
│ Sebastien Tellier │ Kilometer                                            │ Sylvie            │ Cruz             │
├───────────────────┼───────────────────────────────────────────────────

In [14]:
# QUERY 3
# Provide the user name (first and last) of each person that listened to the song 'All Hands Against His Own'

# Create table

query = ("""
CREATE TABLE IF NOT EXISTS user_from_song_listen (
    user_id          INT,
    user_first_name  VARCHAR,
    user_last_name   VARCHAR,
    song_title       VARCHAR,
    PRIMARY KEY (song_title, user_id)
    )
""")
try:
    session.execute(query)
except Exception as exception:
    print(exception)
                    

In [15]:
# QUERY 3
# Provide the user name (first and last) of each person that listened to the song 'All Hands Against His Own'

# Insert data

filepath = 'event_datafile_new.csv'

with open(filepath, encoding = 'utf8') as file:
    csvreader = csv.reader(file)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_from_song_listen (user_id, user_first_name, user_last_name, song_title)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

In [16]:
# QUERY 3
# Provide the user name (first and last) of each person that listened to the song 'All Hands Against His Own'

# Verify data

query = "SELECT user_first_name, user_last_name FROM user_from_song_listen WHERE song_title = 'All Hands Against His Own'"
try:
    query_df = pd.DataFrame(list(session.execute(query)))
except Exception as exception:
    print(exception)

print(tabulate(query_df, headers='keys', tablefmt='fancy_grid', showindex=False))

╒═══════════════════╤══════════════════╕
│ user_first_name   │ user_last_name   │
╞═══════════════════╪══════════════════╡
│ Jacqueline        │ Lynch            │
├───────────────────┼──────────────────┤
│ Tegan             │ Levine           │
├───────────────────┼──────────────────┤
│ Sara              │ Johnson          │
╘═══════════════════╧══════════════════╛


### Drop the tables before closing out the sessions

In [17]:
## Drop the tables before closing out the sessions

drop_queries = [
    "DROP table song_from_session_item",
    "DROP table song_from_user_session",
    "DROP table user_from_song_listen"
]

for query in drop_queries:
    try:
        session.execute(query)
    except Exception as exception:
        print(exception)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()